# Ejecicios 2, 3 y 4: Máquina Enigma 

> Autores: 
> - Andrés Millán Muñoz
> - Juan Antonio Villegas

# Código de la máquina Enigma

## Colección de Rotores

Codificaremos los rotores como diccionarios cuyos campos serán una secuencia de caracteres que representará los efectos en las diferentes letras, y una cadena de uno o dos caracteres que representan el 'notch' (salvo en los casos de las ruedas griegas, los cuartos rotores de la M4, que son fijos).

In [2]:
Ruedas = {
    "base": {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
    },
    "I"  : {
         "secuencia": "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
         "notch"    : "Q",
    },
    "II" : {
         "secuencia": "AJDKSIRUXBLHWTMCQGZNPYFVOE",
         "notch"    : "E",
    },
    "III": {
         "secuencia": "BDFHJLCPRTXVZNYEIWGAKMUSQO",
         "notch"    : "V",
    },
    "IV" : {
         "secuencia": "ESOVPZJAYQUIRHXLNFTGKDCMWB",
         "notch"    : "J",
    },
    "V"  : {
         "secuencia": "VZBRGITYUPSDNHLXAWMJQOFECK",
         "notch"    : "Z",
    },
    "VI" : {
         "secuencia": "JPGVOUMFYQBENHZRDKASXLICTW",
         "notch"    : "ZM",
    },
    "VII": {
         "secuencia": "NZJHGRCXMYSWBOUFAIVLPEKQDT",
         "notch"    : "ZM",
    },
    "VIII": {
         "secuencia": "FKQHTLXOCBJSPDZRAMEWNIUYGV",
         "notch"    : "ZM",
    }
}

Ruedas_griegas = {
    "base" : {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
     },
    "beta" : {
         "secuencia": "LEYJVCNIXWPBQMDRTAKZGFUHOS",
         "notch"    : "",
     },
    "gamma": {
         "secuencia": "FSOKANUERHMBTIYCWLQPZXVGJD",
         "notch"    : "",
     }
}

## Colección de Reflectores
Representamos cada reflector como un par clave-valor donde la clave es la referencia y el valor es una secuencia que al igual que en los rotores representa el cableado interno. Aclaramos que la enigma M4 solo puede usar los reflectores 'B_thin' y 'C_thin', pues eran más finos que los demás, lo que permitía que se pudieran introducir en la caja de la máquina junto con un cuarto rotor.

In [3]:
Reflectores = {
    "base"  : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"     : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"     : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"     : "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    "B_thin": "ENKQAUYWJICOPBLMDXZVFTHRGS",
    "C_thin": "RDOBJNTKVEHMLFCWZAXGYIPSUQ"
}

# Clase Rotor

Encapsula el funcionamiento interno de un rotor. La estructura utilizada es mantener dos secuencias alfabéticas. Una será el alfabeto base pero desplazado, de forma que la primera posición será la que aparecerá en la ventana del rotor en la máquina. La otra será la secuencia correspondiente al rotor, pero desplazada los mismos lugares que la base. 

Un ejemplo que puede ocurrir en cierto momento de una ejecución es, con el rotor I con ringstellung a 'A':
```
GHIJKLMNOPQRSTUVWXYZABCDEF
DQVZNTOWYHXUSPAIBRCJEKMFLG
```
Y en la ventana aparecería la letra 'G'. Si al rotor entra una señal con la letra 'O', saldría una 'Y'.


In [4]:
class Rotor:
    def __init__(self, rueda, ringstellung: str):      
        """
        'rueda' es el mapa de la parte inicial del fichero.
        """

        self.base      = Ruedas["base"]["secuencia"]
        self.secuencia = rueda["secuencia"]
        self.notch     = rueda["notch"]

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        self.__ajusta_ringstellung__(ringstellung)


    #
    # ──────────────────────────────────────────────────────────── CONFIGURACIÓN ─────
    #


    def ajusta_grundstellung(self, indice):
        self.base      = self.base[indice:] + self.base[:indice]
        self.secuencia = self.secuencia[indice:] + self.secuencia[:indice]
    
    
    def __ajusta_ringstellung__(self, ringstellung: str):
        indice_ring = Ruedas["base"]["secuencia"].find(ringstellung)
        
        # Pasaremos temporalmente la secuencia a una lista para poder operarla. 
        self.secuencia = list(self.secuencia)
        
        for i in range(len(self.secuencia)):
            indice = Ruedas["base"]["secuencia"].find(self.secuencia[i]) + indice_ring
            self.secuencia[i] = Ruedas["base"]["secuencia"][indice % 26]
        
        
        cadena = ""
        for c in self.secuencia:
            cadena = cadena + c
        self.secuencia = cadena   
        
        self.secuencia = self.secuencia[-indice_ring:] + self.secuencia[:-indice_ring]


#
# ───────────────────────────────────────────────────────── ACCIÓN DEL ROTOR ─────
#
 
    def siguiente(self, indice: int):
        letra_a_buscar   = self.secuencia[indice]
        indice_siguiente = self.base.index(letra_a_buscar)
        return indice_siguiente


    def revertir(self, indice):
        letra_en_base    = self.base[indice]
        indice_de_salida = self.secuencia.index(letra_en_base)
        return indice_de_salida


    def rotar(self):
        self.base   = self.base[1:] + self.base[:1]
        self.secuencia = self.secuencia[1:] + self.secuencia[:1]


    def ha_tocado_notch(self):
        return self.base[0] in self.notch

## Clase Reflector

In [5]:
class Reflector:
    def __init__(self, reflector: str):
        self.secuencia = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.secuencia[indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

## Clase Plugboard
Representa el zócalo de clavijas. A partir de un array de pares se construye una estructura de datos y un método que intercambia cuando es necesario las letras.

In [6]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base    = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    
    def siguiente(self, letra: str) -> str:
        return self.mapping[letra]

## Clase Enigma
Encapsula todo el funcionamiento de la enigma.

In [7]:
class Enigma:
    def __init__(self, ruedas, ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        # Guardar la configuración inicial para el reseteo
        self.params = [ruedas, ringstellungs, grundstellungs, reflector, steckern]

        # Inicialización de los rotores        
        self.__setup_rotores__(ruedas, ringstellungs, grundstellungs)
        
        # Montamos el resto de los componentes
        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)

        # Los rotores giran cuando se pulsa una letra de una manera determinada. 
        # La enigma M4 utilizó un cuarto rotor que no gira, por lo que el último campo se mantendrá a False. 
        # El primero siempre rota, por lo que constantemente estará a True
        # Utilizar esta estructura nos resultará especialmente útil cuando tengamos que controlar el double step. 
        self.needs_rotation = [False, False, False, True]


    def __setup_rotores__(self, ruedas, ringstellungs: str, grundstellungs: str):
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        # Encontramos los índices correspondientes al grundstellung
        indices = []
        for letra in grundstellungs:
            indices.append(Ruedas["base"]["secuencia"].find(letra))

        for rotor, indice in zip(self.rotores, indices):
            rotor.ajusta_grundstellung(indice)
    
    
    def reset(self):
        self.__setup_rotores__(self.params[0], self.params[1], self.params[2])
        self.needs_rotation = [False, False, False, True]


    #
    # ─────────────────────────────────────────────────────────── FUNCIONAMIENTO ─────
    #


    def __cifrar_caracter__(self, c: str):
        # ────────────────────────────────────────────────── ROTACION ─────
        #
        # Vigilar cuándo toca el notch, para girarlos cuando corresponda. 
        # Utilizar esta estructura nos resultará especialmente útil cuando tengamos que controlar el double step. 

        # Los rotores se iteran de derecha a izquierda:
        # <- [D, C, B, A] <- 
        self.needs_rotation[-2] = self.rotores[-1].ha_tocado_notch()
        self.needs_rotation[-3] = self.rotores[-2].ha_tocado_notch()
        
        # Single step
        for rotor, do_i_rotate in zip(self.rotores[::-1], self.needs_rotation[::-1]):
            if do_i_rotate:
                rotor.rotar()
        
        # Double step
        if self.needs_rotation[-3]:
            self.rotores[-2].rotar()


        # ─────────────────────────────────────────────────── CIFRADO ─────

        # El orden es el siguiente:
        # Pulsamos una tecla de entrada -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)

        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"]["secuencia"].find(letra)

        for rotor in reversed(self.rotores):
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in self.rotores:
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"]["secuencia"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        return letra


    def cifrar(self, mensaje: str):        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.__cifrar_caracter__(letra)
        
        return salida


## Pruebas de ejecución

### Ejemplo 1

Para probar la máquina Enigma, la configuraremos con los requisitos dados por el PDF, y comprobaremos que sale lo debido


In [8]:
rotores   = [Ruedas_griegas["beta"], Ruedas["V"], Ruedas["VI"], Ruedas["VIII"]]
clavijas  = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]
reflector = Reflectores["C_thin"]

ringstellungs            = "EPEL"   
grundstellungs_iniciales = "NAEM"

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)

futuro_grundstellungs = "QEOB"
grundstellungs = enigma.cifrar(futuro_grundstellungs)
print("Grundstellung utilizado: " + grundstellungs)

# Configuramos la nueva Enigma con el grundstellung que hemos obtenido
enigma = Enigma(rotores, ringstellungs, grundstellungs, reflector, clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

# Quitar 2 primeros y 2 últimos grupos, pues se utilizaban para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]

mensaje_cifrado = enigma.cifrar(mensaje)

print(f"Mensaje original: {mensaje}\n")
print(f"Mensaje cifrado: {mensaje_cifrado}\n")

mensaje_museo = "KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP"
mensaje_cifrado.replace(" ", "")
print(f'¿Son iguales el mensaje esperado y el que hemos obtenido? {mensaje_cifrado.replace(" ", "") == mensaje_museo}')


Grundstellung utilizado: CDSZ
Mensaje original: LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW

Mensaje cifrado: KRKR ALLE XXFO LGEN DESI STSO FORT BEKA NNTZ UGEB ENXX ICHH ABEF OLGE LNBE BEFE HLER HALT ENXX JANS TERL EDES BISH ERIG XNRE ICHS MARS CHAL LSJG OERI NGJS ETZT DERF UEHR ERSI EYHV RRGR ZSSA DMIR ALYA LSSE INEN NACH FOLG EREI NXSC HRIF TLSC HEVO LLMA CHTU NTER WEGS XABS OFOR TSOL LENS IESA EMTL ICHE MASS NAHM ENVE RFUE GENY DIES ICHA USDE RGEG ENWA ERTI GENL AGEE RGEB ENXG EZXR EICH SLEI TEIK KTUL PEKK JBOR MANN JXXO BXDX MMMD URNH FKST XKOM XADM XUUU BOOI EXKP

¿So

### Ejemplo 2 de la página wildunix

In [9]:
# Configuración de la Enigma:
rotores = [Ruedas_griegas["beta"], Ruedas["II"], Ruedas['IV'], Ruedas['I']]
reflector = Reflectores["B_thin"]
clavijas = [('A', 'T'), ('B', 'L'), ('D', 'F'), ('G', 'J'), ('H', 'M'), ('N', 'W'), ('O', 'P'), ('Q', 'Y'), ('R', 'Z'), ('V', 'X')]

ringstellungs  = 'AAAV'
grundstellungs = 'VJNA'

enigma = Enigma(rotores, ringstellungs, grundstellungs, reflector, clavijas)

# Descifrado del mensaje
mensaje = 'NCZWVUSXPNYMINHZXMQXSFWXWLKJAHSHNMCOCCAKUQPMKCSMHKSEINJUSBLKIOSXCKUBHMLLXCSJUSRRDVKOHULXWCCBGVLIYXEOAHXRHKKFVDREWEZLXOBAFGYUJQUKGRTVUKAMEURBVEKSUHHVOYHABCJWMAKLFKLMYFVNRIZRVVRTKOFDANJMOLBGFFLEOPRGTFLVRHOWOPBEKVWMUQFMPWPARMFHAGKXIIBG'

mensaje_descifrado = enigma.cifrar(mensaje)

print(f"Mensaje descifrado: {mensaje_descifrado}\n")

mensaje_original = 'VONVONJLOOKSJHFFTTTEINSEINSDREIZWOYYQNNSNEUNINHALTXXBEIANGRIFFUNTERWASSERGEDRUECKTYWABOSXLETZTERGEGNERSTANDNULACHTDREINULUHRMARQUANTONJOTANEUNACHTSEYHSDREIYZWOZWONULGRADYACHTSMYSTOSSENACHXEKNSVIERMBFAELLTYNNNNNNOOOVIERYSICHTEINSNULL'

print(f'¿Son iguales el mensaje esperado y el que hemos obtenido? {mensaje_descifrado == mensaje_original}')

Mensaje descifrado: VONVONJLOOKSJHFFTTTEINSEINSDREIZWOYYQNNSNEUNINHALTXXBEIANGRIFFUNTERWASSERGEDRUECKTYWABOSXLETZTERGEGNERSTANDNULACHTDREINULUHRMARQUANTONJOTANEUNACHTSEYHSDREIYZWOZWONULGRADYACHTSMYSTOSSENACHXEKNSVIERMBFAELLTYNNNNNNOOOVIERYSICHTEINSNULL

¿Son iguales el mensaje esperado y el que hemos obtenido? True


### Ejemplo 3 de wildunix

In [10]:
# Configuración de la Enigma:
rotores  = [Ruedas_griegas["gamma"], Ruedas["IV"], Ruedas["III"], Ruedas["VIII"]]
clavijas = [('C', 'H'), ('E', 'J'), ('N', 'V'), ('O', 'U'), ('T', 'Y'), ('L', 'G'), ('S', 'Z'), ('P', 'K'), ('D', 'I'), ('Q', 'B')]
reflector = Reflectores["B_thin"]

ringstellungs            = "AACU"
grundstellungs_iniciales = "MVYY"

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)

# Descifrado del mensaje
mensaje = "PIARMJTBFRGMEBDSBOGCRVBCOXYOHRWDLQIXQYCXZSKFLQMSIMTQBNZDTXDQIVFUYGLIATLERQYQKSIXMMZUKZTVFCJPPFVGZGOE"

mensaje_descifrado = enigma.cifrar(mensaje)

mensaje_original = 'FFFDDDUUUMSTVINVONHAKFROENNELQAUFGGGZWONULZWONEUXUUBZWOVOMZWOACHTXVSERXXJHRCHTJHIERNICHTEINZELGUFENF'

print(f'¿Son iguales el mensaje esperado y el que hemos obtenido? {mensaje_descifrado == mensaje_original}')

¿Son iguales el mensaje esperado y el que hemos obtenido? True


### Ejemplo M3 

Se puede configurar la máquina para admitir 3 rotores en vez de 4. Por ejemplo...


In [11]:
# Configuración de la Enigma:
rotores  = [Ruedas["VIII"], Ruedas["VI"], Ruedas["V"]]
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]
reflector = Reflectores["C"]

ringstellungs            = "LEP"   
grundstellungs_iniciales = "MEA"   

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)

# Cifrado del mensaje
mensaje = "PRUEBA"
mensaje_cifrado = enigma.cifrar(mensaje)

# Descifrado del mensaje
enigma.reset()
mensaje_descifrado = enigma.cifrar(mensaje_cifrado)

mensaje, mensaje_cifrado, mensaje_descifrado

('PRUEBA', 'WSKUTF', 'PRUEBA')

# Ejercicio 4

El oficial Villegas desde tierra quiere enviarle al oficial Millán, que está en una expedición dentro de un submarino, un mensaje:

> 'Andres, los ingleses os van a meter un misilazo dentro de poco, tened cuidado'.

Que formateado como un texto a enviar por la enigma sería:

> J ANDRES Y LOS INGLESES OS VAN A METER UN MISILAZO DENTRO DE POCO Y TENED CUIDADO JX

Que en bloques de cuatro caracteres sería:

> JAND RESY LOSI NGLE SESO SVAN AMET ERUN MISI LAZO DENT RODE POCO YTEN EDCU IDAD OJX

El oficial Andrés tiene una configuración propia única y conocida solo por los oficiales:
* Zusatzwalze: Beta
* Walzenlage: 241
* Ringstellung: AAAV
* Grundstellung: VJNA
* Steckerverbindungen: AT BL DF GJ HM NW OP QY RZ VX
* UKW: B

Por lo que el oficial Villegas codifica el mensaje con esa configuración, utilizando para el resto de parámetros los indicados por la fecha. Esto es:

In [12]:
rotores = [Ruedas_griegas["beta"], Ruedas["II"], Ruedas['IV'], Ruedas['I']]
reflector = Reflectores["B_thin"]
clavijas = [('A', 'T'), ('B', 'L'), ('D', 'F'), ('G', 'J'), ('H', 'M'), ('N', 'W'), ('O', 'P'), ('Q', 'Y'), ('R', 'Z'), ('V', 'X')]

ringstellungs  = 'AAAV'
grundstellungs = 'VJNA'

enigma1 = Enigma(rotores, ringstellungs, grundstellungs, reflector, clavijas)

Ahora el oficial Villegas encripta su mensaje con esta configuración de enigma, 

In [13]:
mensaje = "JAND RESY LOSI NGLE SESO SVAN AMET ERUN MISI LAZO DENT RODE POCO YTEN EDCU IDAD OJX"
mensaje_destinatario = enigma1.cifrar(mensaje)
mensaje_destinatario

'ZTZT KIJC UNPW DLXT EYJF IPEV CACA ZYVP XYUL XEVF CQWY NHKW FIKU XBTF PPOH FRJV ICT'

Y ahora se añade el destinatario al principio del mensaje:

In [14]:
mensaje_a_cifrar = "OFIC IALA NDRE SMIL LANX " + mensaje_destinatario
mensaje_a_cifrar

'OFIC IALA NDRE SMIL LANX ZTZT KIJC UNPW DLXT EYJF IPEV CACA ZYVP XYUL XEVF CQWY NHKW FIKU XBTF PPOH FRJV ICT'

A continuación el oficial Villegas encripta el mensaje con la configuración de ese día:

* Zusatzwalze: Gamma
* Walzenlage: 438
* Ringstellung: AACU
* Grundstellung: MVYY
* Steckerverbindungen: CH EJ NV OU TY LG SZ PK DI QB
* UKW: B

Y añade los caracteres de verificación, que por analogía con el ejercicio 2 seran "DUHF TETO"

In [15]:
rotores  = [Ruedas_griegas["gamma"], Ruedas["IV"], Ruedas["III"], Ruedas["VIII"]]
clavijas = [('C', 'H'), ('E', 'J'), ('N', 'V'), ('O', 'U'), ('T', 'Y'), ('L', 'G'), ('S', 'Z'), ('P', 'K'), ('D', 'I'), ('Q', 'B')]
reflector = Reflectores["B_thin"]

ringstellungs  = "AACU"
grundstellungs = "MVYY"

verificacion = "DUHF TETO"

enigma2 = Enigma(rotores, ringstellungs, grundstellungs, reflector, clavijas)
mensaje_enviado = verificacion + ' ' + enigma2.cifrar(mensaje_a_cifrar) + ' ' + verificacion

mensaje_enviado 

'DUHF TETO ZIZV VPSY BAJQ TPGM FVDW JKYZ PAIV MWSI BNBS OWWL RZBG ZYOG KFLE TOKK DZPL VOTQ HONY PVEH FGZT WYCX VWXD KZH DUHF TETO'

Y ese es el mensaje que se envía. El oficial de llaves recibe ese mensaje desde dentro del submarino, configura la enigma con las claves de ese día y desencripta el mensaje.

In [16]:
mensaje_recibido = mensaje_enviado[10:-10] # Retiramos los caracteres de verificacion
print(mensaje_recibido)
enigma2.reset()
mensaje_desencriptado = enigma2.cifrar(mensaje_recibido)
mensaje_desencriptado

ZIZV VPSY BAJQ TPGM FVDW JKYZ PAIV MWSI BNBS OWWL RZBG ZYOG KFLE TOKK DZPL VOTQ HONY PVEH FGZT WYCX VWXD KZH


'OFIC IALA NDRE SMIL LANX ZTZT KIJC UNPW DLXT EYJF IPEV CACA ZYVP XYUL XEVF CQWY NHKW FIKU XBTF PPOH FRJV ICT'

Y aquí el oficial de llaves ve que el mensaje está dirigido al oficial Andrés Millán, el cual con sus claves privadas desencripta el mensaje recibido:

In [17]:
enigma1.reset()
mensaje_privado = mensaje_desencriptado[25:]    # Retiramos el destinatario
mensaje_final = enigma1.cifrar(mensaje_privado)

mensaje_final

'JAND RESY LOSI NGLE SESO SVAN AMET ERUN MISI LAZO DENT RODE POCO YTEN EDCU IDAD OJX'

Y así el oficial MIllán consiguió desviar la trayectoria del submarino, salvando su vida y la de su tripulación.

In [18]:
mensaje_final == mensaje

True